In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import time #for test and progress
#from streamlit_extras.dataframe_explorer import dataframe_explorer
import matplotlib.pyplot as plt
import seaborn as sb
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import plotly.express as px

In [3]:
final_df = pd.read_json("final_json.json")

In [4]:
final_df

,object,id,oracle_id,multiverse_ids,name,lang,released_at,uri,scryfall_uri,layout,...,rulings,subtypes,supertypes,text,type,types,uuid,edhrecRank,leadershipSkills,prices
0,card,dbcdbf7a-9294-47ad-9f93-c16b78c7463a,cd6250ae-9079-4a62-8a70-0d94fbac21bc,[-1],Earthshaker Giant,en,2019-11-15,https://api.scryfall.com/cards/dbcdbf7a-9294-4...,https://scryfall.com/card/gn2/5/earthshaker-gi...,normal,...,[no additional rules],"[Giant, Druid]",[no supertypes],Trample\nWhen Earthshaker Giant enters the bat...,Creature — Giant Druid,[Creature],d77da7e3-b017-559f-8081-eb56d48da8a2,not ranked,unusable,"{'mtgo': {}, 'mtgoFoil': {}, 'paper': {}, 'pap..."
1,card,acb3ce9b-ee4f-410a-8db3-e87aeb0a4444,ab0dfae5-b9d4-417b-8a0d-2525ae3a73b9,[-1],Fiendish Duo,en,2019-11-15,https://api.scryfall.com/cards/acb3ce9b-ee4f-4...,https://scryfall.com/card/gn2/4/fiendish-duo?u...,normal,...,[no additional rules],[Devil],[no supertypes],First strike\nIf a source would deal damage to...,Creature — Devil,[Creature],dcb1e7e2-bd48-5d9b-a872-7386e81bbac4,not ranked,unusable,"{'mtgo': {}, 'mtgoFoil': {}, 'paper': {}, 'pap..."
2,card,17b2ed72-d0f0-4d8d-bb5e-dce08d157466,7264d3b3-fd46-4ea3-a85d-f0b068c331ad,[-1],Calculating Lich,en,2019-11-15,https://api.scryfall.com/cards/17b2ed72-d0f0-4...,https://scryfall.com/card/gn2/3/calculating-li...,normal,...,[no additional rules],"[Zombie, Wizard]",[no supertypes],Menace\nWhenever a creature attacks one of you...,Creature — Zombie Wizard,[Creature],aedff6d1-7901-5619-9ff6-b3dd9c43b896,not ranked,unusable,"{'mtgo': {}, 'mtgoFoil': {}, 'paper': {}, 'pap..."
3,card,0faa9eea-fbf1-41f7-9def-1ec3d5134a53,e6284fb3-7cf8-4730-b156-10085b70b0e8,[-1],Sphinx of Enlightenment,en,2019-11-15,https://api.scryfall.com/cards/0faa9eea-fbf1-4...,https://scryfall.com/card/gn2/2/sphinx-of-enli...,normal,...,[no additional rules],[Sphinx],[no supertypes],Flying\nWhen Sphinx of Enlightenment enters th...,Creature — Sphinx,[Creature],5c885aa6-c991-5eec-b140-d43a70428566,not ranked,unusable,"{'mtgo': {}, 'mtgoFoil': {}, 'paper': {}, 'pap..."
4,card,ecbeac44-9392-4522-8ff5-87079386bd0a,43296f8b-58d9-446e-a538-1c4921552c41,[-1],Highcliff Felidar,en,2019-11-15,https://api.scryfall.com/cards/ecbeac44-9392-4...,https://scryfall.com/card/gn2/1/highcliff-feli...,normal,...,[no additional rules],"[Cat, Beast]",[no supertypes],Vigilance\nWhen Highcliff Felidar enters the b...,Creature — Cat Beast,[Creature],a9bf68a1-7eb7-5387-a651-5c0ede8d86ec,not ranked,unusable,"{'mtgo': {}, 'mtgoFoil': {}, 'paper': {}, 'pap..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24294,card,d2379f78-c03f-447f-b3c9-10a918d556e9,4627691c-4ed4-4add-9cc3-2e019be2f9fd,[245],Consecrate Land,en,1993-08-05,https://api.scryfall.com/cards/d2379f78-c03f-4...,https://scryfall.com/card/lea/14/consecrate-la...,normal,...,"[{'date': '2005-04-01', 'text': 'The land can ...",[Aura],[no supertypes],Enchant land\nEnchanted land has indestructibl...,Enchantment — Aura,[Enchantment],71185011-e055-5986-8825-4acac9041b4f,13595,unusable,"{'mtgo': {}, 'mtgoFoil': {}, 'paper': {'2019-1..."
24295,card,93f9f0f2-e1cc-4740-888c-1336c6de0a27,fc0bf1d0-46a2-4305-ab2e-466d79d60ab2,[239],Blue Ward,en,1993-08-05,https://api.scryfall.com/cards/93f9f0f2-e1cc-4...,https://scryfall.com/card/lea/8/blue-ward?utm_...,normal,...,[no additional rules],[Aura],[no supertypes],Enchant creature\nEnchanted creature has prote...,Enchantment — Aura,[Enchantment],cbde4259-23f6-5ee8-89a9-0d73647e49e0,12313,unusable,"{'mtgo': {}, 'mtgoFoil': {}, 'paper': {}, 'pap..."
24296,card,98fba951-c5bb-497c-9292-ce1b2a1e1247,b330ac89-790e-4cc9-96a5-532c48252088,[237],Blaze of Glory,en,1993-08-05,https://api.scryfall.com/cards/98fba951-c5bb-4...,https://scryfall.com/card/lea/6/blaze-of-glory...,normal,...,"[{'date': '2004-10-04', 'text': 'Does not allo...",[no subtypes],[no supertypes],Cast this spell only during combat before bloc...,Instant,[Instant],68549548-8ec2-5232-984f-1064ae9ff59b,16228,unusable,

In [6]:
lista_ind_paper = []
for ind in final_df.index:
    if bool(final_df.prices.iloc[ind]['paper']) == True: #prendo solo quelli che paper, in teoria
        lista_ind_paper.append(ind)
df_prices_paper = final_df.iloc[lista_ind_paper]
#ma se creassi una colonna apposta, mi sembra più semplice
#creo una serie
serie_paper = pd.Series()
lista_prezzi_paper = []
df_prices_paper.reset_index(inplace=True)

for ind in df_prices_paper.index:
    prezzo = df_prices_paper.prices.iloc[ind]['paper']
    valore_prezzo = prezzo.get('2019-11-09')
    lista_prezzi_paper.append(valore_prezzo)

df_prices_paper['price_paper'] = lista_prezzi_paper
df_prices_paper_unique = df_prices_paper.drop_duplicates('name')
prova_predict = df_prices_paper_unique.copy()

#corr_prova_predict = st.checkbox('Show prova')

#if corr_prova_predict:
#    fig,ax = plt.subplots()
#    ax = sb.heatmap(prova_predict.corr(numeric_only=True),annot=True, fmt=".1f", linewidth=.5)
#    st.pyplot(fig)

#from list/dict to string
list_colonne_to_string = ['printings','legalities']

for x in list_colonne_to_string:
    stringa = prova_predict[x].astype(str)
    col=x+'_string'
    prova_predict[col] = stringa

#df per prediction
pred = prova_predict[['rarity','price_paper','cmc','power','toughness','reprint','printings_string','set_name','legalities_string','reserved']]
#creo a categorical variable for rarity
categorical_feature = ['rarity','power','toughness','reserved','reprint','printings_string','legalities_string','set_name'] #ok accetta solo stringhe o numeri, devo trasformare le liste e disct in stringhe
le = LabelEncoder()

#convert the variable to some sort of numerical
for i in range(8):
    new = le.fit_transform(pred[categorical_feature[i]])
    pred[categorical_feature[i]] = new

pred.dropna(inplace=True)

#training the model
column_test = ['rarity','cmc','power','toughness','reprint','printings_string','set_name','legalities_string','reserved']
#column_test = ['rarity','cmc','power','toughness','reprint','set_name','reserved']
X = pred[column_test]
y = pred.price_paper

#splitting data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,test_size=0.2, random_state=0) #capire test size e random state

#train the model
#regr = RandomForestRegressor(n_estimators=100, max_depth=10, random_state = 101) #capire
regr = RandomForestRegressor(n_estimators=100, min_samples_split=2, min_samples_leaf=1, bootstrap=True,  oob_score=True) #capire, disolito negli esmpi mettono n_estimators come 10, max_depth non ce e random_state = 0
regr.fit(X_train, y_train.values.ravel())

#make prediction
warnings.filterwarnings('ignore')
predictions = regr.predict(X_test)
result = X_test
result['paper_price'] = y_test
result['prediction'] = predictions.tolist()

#errori/risultato test
#model evaluation
mse = mean_squared_error(y_test.values.ravel(), predictions)
r2 = r2_score(y_test.values.ravel(), predictions)
mae = mean_absolute_error(y_test.values.ravel(), predictions)

#st.write(result)
print("Mean square error (MSE):", round(mse, 2))
print("R2 Score: ", round(r2, 2))
print("Mean Absolute Error (MAE): ", round(mae, 2))
oob_score = regr.oob_score_
print("Out-of-Bag Score: ", round(oob_score, 2))


Mean square error (MSE): 420.47
R2 Score:  0.28
Mean Absolute Error (MAE):  1.95
Out-of-Bag Score:  0.34


In [26]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
lista_ind_paper = []
for ind in final_df.index:
    if bool(final_df.prices.iloc[ind]['paper']) == True: #prendo solo quelli che paper, in teoria
        lista_ind_paper.append(ind)
df_prices_paper = final_df.iloc[lista_ind_paper]
#ma se creassi una colonna apposta, mi sembra più semplice
#creo una serie
serie_paper = pd.Series()
lista_prezzi_paper = []
df_prices_paper.reset_index(inplace=True)

for ind in df_prices_paper.index:
    prezzo = df_prices_paper.prices.iloc[ind]['paper']
    valore_prezzo = prezzo.get('2019-11-09')
    lista_prezzi_paper.append(valore_prezzo)

df_prices_paper['price_paper'] = lista_prezzi_paper
df_prices_paper_unique = df_prices_paper.drop_duplicates('name')
prova_predict = df_prices_paper_unique.copy()

#corr_prova_predict = st.checkbox('Show prova')

#if corr_prova_predict:
#    fig,ax = plt.subplots()
#    ax = sb.heatmap(prova_predict.corr(numeric_only=True),annot=True, fmt=".1f", linewidth=.5)
#    st.pyplot(fig)

#from list/dict to string
list_colonne_to_string = ['printings','legalities']

for x in list_colonne_to_string:
    stringa = prova_predict[x].astype(str)
    col=x+'_string'
    prova_predict[col] = stringa

#df per prediction
pred = prova_predict[['rarity','price_paper','cmc','power','toughness','reprint','printings_string','set_name','legalities_string','reserved']]
#creo a categorical variable for rarity
categorical_feature = ['rarity','power','toughness','reserved','reprint','printings_string','legalities_string','set_name'] #ok accetta solo stringhe o numeri, devo trasformare le liste e disct in stringhe
le = LabelEncoder()

#convert the variable to some sort of numerical
for i in range(8):
    new = le.fit_transform(pred[categorical_feature[i]])
    pred[categorical_feature[i]] = new

pred.dropna(inplace=True)

#training the model
column_test = ['rarity','cmc','power','toughness','reprint','printings_string','set_name','legalities_string','reserved']
X = pred[column_test]
y = pred.price_paper

#splitting data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,test_size=0.2, random_state=0) #capire test size e random state

#train the model
#regr = RandomForestRegressor(n_estimators=100, max_depth=10, random_state = 101) #capire
regr = RandomForestRegressor() #capire, disolito negli esmpi mettono n_estimators come 10, max_depth non ce e random_state = 0
regr_random = RandomizedSearchCV(estimator = regr, param_distributions = random_grid, n_iter= 100, cv = 3, verbose = 2, random_state=42, n_jobs = -1)
#regr.fit(X_train, y_train.values.ravel())
regr_random.fit(X_train, y_train)

#make prediction
warnings.filterwarnings('ignore')
predictions = regr_random.predict(X_test)
result = X_test
result['paper_price'] = y_test
result['prediction'] = predictions.tolist()

#errori/risultato test
#model evaluation
mse = mean_squared_error(y_test.values.ravel(), predictions)
r2 = r2_score(y_test.values.ravel(), predictions)
mae = mean_absolute_error(y_test.values.ravel(), predictions)

#st.write(result)
print("Mean square error (MSE):", round(mse, 2))
print("R2 Score: ", round(r2, 2))
print("Mean Absolute Error (MAE): ", round(mae, 2))
oob_score = regr.oob_score_
print("Out-of-Bag Score: ", round(oob_score, 2))


In [5]:
result

,rarity,price_paper,cmc,power,toughness,reprint,printings_string,set_name,legalities_string,reserved,paper_price,prediction
13075,2,0.76,4.0,23,25,0,4098,213,48,0,0.76,0.764298
19052,3,0.20,4.0,19,22,0,1515,257,82,0,0.20,0.200062
11731,2,0.25,7.0,28,31,0,1039,250,48,0,0.25,0.247835
14382,0,0.06,3.0,28,31,0,1496,24,42,0,0.06,0.062531
10931,1,5.76,6.0,23,26,0,4263,286,53,0,5.76,5.759351
...,...,...,...,...,...,...,...,...,...,...,...,...
14791,0,0.09,2.0,17,11,0,2369,28,42,0,0.09,0.091704
14463,0,0.12,4.0,17,22,0,1496,24,42,0,0.12,0.118322
19956,0,0.06,4.0,10,22,0,3918,194,77,0,0.06,0.062531
20856,2,210.09,3.0,17,20,0,3684,160,86,1,210.09,206.156100
